In [1]:
from google.colab import output
output.enable_custom_widget_manager()

In [2]:
#install library
!pip install -q transformers datasets accelerate rouge-score nltk rouge
!pip install --upgrade datasets
!pip install -q bert-score

#import library
import os
import torch
import nltk
from transformers import T5ForConditionalGeneration, T5Tokenizer, Trainer, TrainingArguments
from datasets import load_dataset
import numpy as np
from rouge import Rouge
from nltk.tokenize import word_tokenize
from nltk.translate.meteor_score import meteor_score
from bert_score import score as bert_score

nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 62.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 36.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 93.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 16.6 MB/s eta 0:00:00
 

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [3]:
# Mencegah disconnect di Colab walau sepertinya tidak berguna jadi manual pakai console javascript
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True,garbage_collection_threshold:0.8"

def prevent_disconnect(interval=300):
    import time, requests
    while True:
        time.sleep(interval)
        requests.get("https://www.google.com")

import threading
keep_alive = threading.Thread(target=prevent_disconnect, daemon=True)
keep_alive.start()

In [4]:
import re

def clean_text(text):
    # Clean url/link
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    # Clean spasi berlebih
    text = re.sub(r'\s+', ' ', text).strip()

    return text


In [5]:
# Load dataset
dataset = load_dataset("cnn_dailymail", "3.0.0")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/15.6k [00:00<?, ?B/s]

train-00000-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00001-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00002-of-00003.parquet:   0%|          | 0.00/259M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/34.7M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/30.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

In [6]:
dataset["train"].select(range(5))

Dataset({
    features: ['article', 'highlights', 'id'],
    num_rows: 5
})

In [7]:
# Load T5 model dan tokenizer
model_name = "google/flan-t5-base"
model = T5ForConditionalGeneration.from_pretrained(model_name)
tokenizer = T5Tokenizer.from_pretrained(model_name)

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [8]:
print(model)

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear(in_features=768, out_features=2048, bias=False)
              (wo):

In [9]:
# Subset data
dataset['train'] = dataset['train'].select(range(10000))
dataset['validation'] = dataset['validation'].select(range(3000))
dataset['test'] = dataset['test'].select(range(3000))

# fungsi preprocessing
def preprocess_function(examples):
    cleaned_articles = [clean_text(doc) for doc in examples['article']]
    cleaned_highlights = [clean_text(hl) for hl in examples['highlights']]

    inputs = ["summarize: " + doc for doc in cleaned_articles]

    model_inputs = tokenizer(
        inputs,
        max_length=512,
        truncation=True,
        padding='max_length'
    )

    labels = tokenizer(
        text_target=cleaned_highlights,
        max_length=128,
        truncation=True,
        padding='max_length'
    )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Mapping dengan multiprocessing (jika stabil)
encoded_dataset = dataset.map(
    preprocess_function,
    batched=True,
    num_proc=2  # atau hapus dulu jika belum stabil
)


Map (num_proc=2):   0%|          | 0/10000 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/3000 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/3000 [00:00<?, ? examples/s]

In [10]:
# Optional: Subset kecil untuk debug cepat
train_dataset = encoded_dataset["train"].select(range(10000))
test_dataset = encoded_dataset["validation"].select(range(3000))


In [12]:
training_args = TrainingArguments(
    output_dir="./flant5_results",
    eval_strategy="epoch",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    learning_rate=3e-5,
    logging_steps=100,
    num_train_epochs=3,
    report_to="none",
    weight_decay=0.01,
    save_total_limit=3,
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)


trainer.train()


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss
1,0.986400,0.858200
2,0.960600,0.861391
3,0.927000,0.864644


TrainOutput(global_step=15000, training_loss=1.0506614125569662, metrics={'train_runtime': 9287.5736, 'train_samples_per_second': 3.23, 'train_steps_per_second': 1.615, 'total_flos': 2.054272057344e+16, 'train_loss': 1.0506614125569662, 'epoch': 3.0})

In [13]:
# Evaluasi
eval_results = trainer.evaluate()
print(f"Evaluation results: {eval_results}")

Evaluation results: {'eval_loss': 0.8646438717842102, 'eval_runtime': 186.699, 'eval_samples_per_second': 16.069, 'eval_steps_per_second': 8.034, 'epoch': 3.0}


In [14]:
# Generate summaries
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def generate_summary_batch(batch):
    with torch.no_grad():
        inputs = tokenizer(
            ["summarize: " + text for text in batch["article"]],
            padding=True,
            truncation=True,
            max_length=512,
            return_tensors="pt"
        ).to(device)

        # Generate summaries with diverse beam search
        outputs = model.generate(
            inputs["input_ids"],
            max_length=128,
            num_beams=4,
            length_penalty=2.0,
            no_repeat_ngram_size=3,
            early_stopping=True
        )

        summaries = tokenizer.batch_decode(outputs, skip_special_tokens=True)
        torch.cuda.empty_cache()

        return {"summary": summaries}

# Generate summaries in batches
summaries = test_dataset.map(generate_summary_batch, batched=True, batch_size=4)

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

In [15]:
# Print beberapa contoh summary
for i in range(3):
    print(f"\nExample {i+1}:")
    print("Article:", test_dataset[i]["article"][:200] + "...")
    print("\nReference Summary:", test_dataset[i]["highlights"])
    print("\nGenerated Summary:", summaries[i]["summary"])
    print("-" * 50)

# Calculate ROUGE scores
def calculate_rouge(references, predictions):
    rouge = Rouge()
    scores = rouge.get_scores(predictions, references, avg=True)
    return {
        "rouge-1": scores["rouge-1"]["f"],
        "rouge-2": scores["rouge-2"]["f"],
        "rouge-l": scores["rouge-l"]["f"]
    }

# Calculate METEOR scores
def calculate_meteor_score(references, predictions):
    total_score = 0.0
    count = 0
    for ref, pred in zip(references, predictions):
        try:
            ref_tokens = [word_tokenize(ref)]
            pred_tokens = word_tokenize(pred)
            score = meteor_score(ref_tokens, pred_tokens)
            total_score += score
            count += 1
        except:
            continue  # Lewati jika error
    return total_score / count if count > 0 else 0.0

# Calculate BERTScore
def calculate_bertscore(references, predictions):
    P, R, F1 = bert_score(predictions, references, lang="en", rescale_with_baseline=True)
    return {
        "bertscore_precision": P.mean().item(),
        "bertscore_recall": R.mean().item(),
        "bertscore_f1": F1.mean().item()
    }

rouge_scores = calculate_rouge(
    [ex["highlights"] for ex in test_dataset],
    [ex["summary"] for ex in summaries]
)

print("\nROUGE Scores:")
print(f"ROUGE-1: {rouge_scores['rouge-1']:.4f}")
print(f"ROUGE-2: {rouge_scores['rouge-2']:.4f}")
print(f"ROUGE-L: {rouge_scores['rouge-l']:.4f}")

# METEOR Score
meteor = calculate_meteor_score(
    [ex["highlights"] for ex in test_dataset],
    [ex["summary"] for ex in summaries]
)
print("METEOR Score:", meteor)

# BERTScore
bertscore_result = calculate_bertscore(
    [ex["highlights"] for ex in test_dataset],
    [ex["summary"] for ex in summaries]
)
print("\nBERTScore:")
print(f"Precision: {bertscore_result['bertscore_precision']:.4f}")
print(f"Recall:    {bertscore_result['bertscore_recall']:.4f}")
print(f"F1 Score:  {bertscore_result['bertscore_f1']:.4f}")


Example 1:
Article: (CNN)Share, and your gift will be multiplied. That may sound like an esoteric adage, but when Zully Broussard selflessly decided to give one of her kidneys to a stranger, her generosity paired up with...

Reference Summary: Zully Broussard decided to give a kidney to a stranger .
A new computer program helped her donation spur transplants for six kidney patients .

Generated Summary: Zully Broussard gave one of her kidneys to a stranger . It resulted in six patients receiving transplants . "I thought I was going to help this one person who I don't know, but the fact that so many people can have a life extension, that's pretty big," she told CNN affiliate KGO . The power that multiplied her gift was data processing of donor-recipient pairs .
--------------------------------------------------

Example 2:
Article: (CNN)On the 6th of April 1996, San Jose Clash and DC United strode out in front of 31,683 expectant fans at the Spartan Stadium in San Jose, California. The

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



BERTScore:
Precision: 0.2427
Recall:    0.2720
F1 Score:  0.2576


In [16]:
import shutil
# Save final model jadikan zip
trainer.save_model("./Flant5_result/final_model")
# Simpan tokenizer ke folder model
tokenizer.save_pretrained("./Flant5_result/final_model")
shutil.make_archive("Flant5_result", 'zip', "./Flant5_result/final_model")
# save model ke drive
from google.colab import drive
drive.mount('/content/drive')
!mkdir -p "/content/drive/MyDrive/Colab Notebooks/FlanT5FIX"
!cp Flant5_result.zip "/content/drive/MyDrive/Colab Notebooks/FlanT5FIX/"

Mounted at /content/drive
